In [78]:
import json, time, requests, timeit, pandas as pd, re

In [4]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
api_url = 'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=1&offset=1&region=north_america'
%timeit api = requests.get(api_url, headers=headers)


0.5079059600830078 s


In [5]:
data = json.loads(api.text)

In [8]:
data

{'data': {'filters': None,
  'table': {'headers': {'symbol': 'Symbol',
    'name': 'Name',
    'lastsale': 'Last Sale',
    'netchange': 'Net Change',
    'pctchange': '% Change',
    'marketCap': 'Market Cap'},
   'rows': [{'symbol': 'MSFT',
     'name': 'Microsoft Corporation Common Stock',
     'lastsale': '$249.68',
     'netchange': '0.37',
     'pctchange': '0.148%',
     'marketCap': '1,880,483,533,894',
     'url': '/market-activity/stocks/msft'}]},
  'totalrecords': 6162,
  'asof': 'Last price as of May 28, 2021'},
 'message': None,
 'status': {'rCode': 200, 'bCodeMessage': None, 'developerMessage': None}}

In [31]:
%timeit num_of_listed_comp = data['data']['totalrecords']

66.3 ns ± 1.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [32]:
%timeit num_of_listed_comp = data.get('data').get('totalrecords') # which is faster?

107 ns ± 2.28 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [33]:
print(num_of_listed_comp)

6162


In [35]:
# 주가 데이터
stock_data = data.get('data').get('table').get('rows')[0]
print(type(stock_data))
print(stock_data)

<class 'dict'>
{'symbol': 'MSFT', 'name': 'Microsoft Corporation Common Stock', 'lastsale': '$249.68', 'netchange': '0.37', 'pctchange': '0.148%', 'marketCap': '1,880,483,533,894', 'url': '/market-activity/stocks/msft'}


In [161]:
df = pd.DataFrame(stock_data, index=[0])

In [162]:
df

,symbol,name,lastsale,netchange,pctchange,marketCap,url
0,MSFT,Microsoft Corporation Common Stock,$249.68,0.37,0.148%,"1,880,483,533,894",/market-activity/stocks/msft


In [171]:
# 형변환
# 1
# lastsale 데이터에 $ 표시 지우기
df['lastsale'] = df['lastsale'].apply(lambda x : re.sub('^\$', '', x))
df['lastsale'] = df['lastsale'].astype(float)

In [164]:
# 2
df['netchange'] = df['netchange'].astype(float)

'$249.68'

In [173]:
# 3
# pctchange 데이터에 % 표시 지우기
df['pctchange'] = df['pctchange'].apply(lambda x : re.sub('%', '', x))
df['pctchange'] = df['pctchange'].astype(float)

In [175]:
# 4
# marketCap에 , 표시 지우기
df['marketCap'] = df['marketCap'].apply(lambda x : re.sub(',', '', x))
df['marketCap'] = df['marketCap'].astype(int)

0    1880483533894
Name: marketCap, dtype: int64